In [51]:

#import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
#import numpy as np
from selenium.webdriver.support.ui import Select

s = Service(ChromeDriverManager().install())
option = webdriver.ChromeOptions()
option.add_argument('headless')
#option.add_argument("--window-size=1920,1080")  # Adjust window size
#option.add_argument("--disable-extensions")  # Disable Chrome extensions

#path = "C:\Users\dillonk\.cache\selenium\chromedriver\win64\124.0.6367.207\chromedriver.exe"
driver = webdriver.Chrome(service = s, options=option)


In [52]:

from selenium.common.exceptions import TimeoutException, WebDriverException

url_oddstrader = "https://www.oddstrader.com/major-league-soccer/"

try:
    driver.get(url_oddstrader)
except WebDriverException as e:
    print(f"An error occurred while loading the page: {e}")
    # Perform appropriate error handling or logging here

try:
    ot_all_teams_list = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="teamName blueHover"]'))
    )

    day_of_game = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.XPATH, '//span[@class="generalDay"]'))
    )

    ot_away_teams_list = []
    ot_home_teams_list = []
    ot_draw_counter = []

    def gameday_checker(day_element):
        gamedays = {"SUN", "MON", "TUE", "WED", "THU", "FRI", "SAT"}
        return any(day in day_element.text for day in gamedays)

    team_index = 0
    # Iterate over the days to filter and collect team names
    # Iterate over the days to filter and collect team names
    for _ in range(len(day_of_game)):
        day_element = day_of_game[_]
        if gameday_checker(day_element):
            if team_index < len(ot_all_teams_list) - 1:
                ot_away_teams_list.append(ot_all_teams_list[team_index].text)
                ot_home_teams_list.append(ot_all_teams_list[team_index + 1].text)
                ot_draw_counter.append(ot_all_teams_list[team_index + 2].text)
            team_index += 3

    # Print the lists of away and home teams
    print("Away Teams:", ot_away_teams_list)
    print("Home Teams:", ot_home_teams_list)

except TimeoutException as te:
    print("Timeout occurred while waiting for elements:", te)
    # Log page source for debugging

print(len(ot_away_teams_list))
print(len(ot_home_teams_list))


Timeout occurred while waiting for elements: Message: 

13
13


In [53]:
#ODDSTRADER (BEST ODDS LISTED): 

ot_all_odds_list = []
ot_best_home_odds_list = []
ot_best_away_odds_list = []
ot_best_draw_odds_list = []


ot_all_odds_list = WebDriverWait(driver, 3).until(
    EC.presence_of_all_elements_located((By.XPATH, '//span[@class="best-line"]'))
)

i = 0
while i < len(ot_away_teams_list) + len(ot_home_teams_list) + len(ot_draw_counter):
    if i % 3 == 0:
        ot_best_away_odds_list.append(ot_all_odds_list[i].text)
    elif i % 3 == 1:
        ot_best_home_odds_list.append(ot_all_odds_list[i].text)
    elif i %3 ==2:
        ot_best_draw_odds_list.append(ot_all_odds_list[i].text)
    i += 1


# Convert US odds to UK odds
def convert_odds_to_uk(us_odds_list):
    uk_odds_list = []
    for us_odd in us_odds_list:
        if us_odd == 'even':
            uk_odd = 2.0
        elif us_odd == '-':
            uk_odd = 1.0  # Representing a loss of the bet as 100% loss (decimal odds of 1.0)
        else:
            us_odd = int(us_odd)
            if us_odd > 0:
                uk_odd = (us_odd / 100) + 1
            else:
                uk_odd = (100 / abs(us_odd)) + 1
        uk_odds_list.append(round(uk_odd, 2))
    return uk_odds_list

ot_best_away_odds_us = ot_best_away_odds_list.copy()
ot_best_home_odds_us = ot_best_home_odds_list.copy()
ot_best_draw_odds_us = ot_best_draw_odds_list.copy()

ot_best_away_odds_uk = convert_odds_to_uk(ot_best_away_odds_list)
ot_best_home_odds_uk = convert_odds_to_uk(ot_best_home_odds_list)
ot_best_draw_odds_uk = convert_odds_to_uk(ot_best_draw_odds_list)

# Print odds data

print("Away Odds (US):", ot_best_away_odds_us)
print("Home Odds (US):", ot_best_home_odds_us)
print("Draw Odds (US):", ot_best_draw_odds_us)

print(len(ot_best_away_odds_uk))
print(len(ot_best_home_odds_uk))
print(len(ot_best_draw_odds_uk))

TimeoutException: Message: 


In [ ]:
#odds trader best sites: 


from selenium.common.exceptions import StaleElementReferenceException

ot_all_best_sites = []
ot_best_away_sites = []
ot_best_home_sites = []
ot_best_draw_sites = []

# Wait for all elements to be present
ot_all_best_sites = WebDriverWait(driver, 7).until(
    EC.presence_of_all_elements_located((By.XPATH, '//span[@class="subtitle"]'))
)


retry_count = 5
while retry_count > 0:
    try:
        ot_all_best_sites_text = [elem.text for elem in ot_all_best_sites if elem.text.strip() != "ODDS BY"]
        # Slice the list to separate away, home, and draw odds
        ot_best_away_sites = [site for site in ot_all_best_sites_text[::3] if site.strip()]
        ot_best_home_sites = [site for site in ot_all_best_sites_text[1::3] if site.strip()]
        ot_best_draw_sites = [site for site in ot_all_best_sites_text[2::3] if site.strip()]

        break  # Exit the loop if successful
    except StaleElementReferenceException:
        retry_count -= 1
if retry_count == 0:
    print("Failed to retrieve element text due to StaleElementReferenceException")

# Print the best sites lists
print("Best Away Sites:", ot_best_away_sites)
print("Best Home Sites:", ot_best_home_sites)
print("Best Draw Sites:", ot_best_draw_sites)



Best Away Sites: ['DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings']
Best Home Sites: ['DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings']
Best Draw Sites: ['DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings']


In [ ]:
#coverting uk odds from oddstrader into away prob and home prob 

ot_home_probablity_odds = []
for odds in ot_best_home_odds_uk: 
    ot_home_probablity_odds.append(1 /odds)

ot_away_probablity_odds = []
for odds in ot_best_away_odds_uk: 
    ot_away_probablity_odds.append(1 /odds)

ot_draw_probablility_odds = []
for odds in ot_best_draw_odds_uk:
    ot_draw_probablility_odds.append(1/ odds)

print(ot_away_probablity_odds)
print(ot_home_probablity_odds)
print(ot_draw_probablility_odds)

[0.4166666666666667, 0.4081632653061224, 0.6060606060606061, 0.4166666666666667, 0.4166666666666667, 0.45454545454545453, 0.36363636363636365, 0.36363636363636365, 0.45454545454545453, 0.5128205128205129, 0.4081632653061224, 0.5649717514124294, 0.53475935828877]
[0.4, 0.37735849056603776, 0.22727272727272727, 0.37037037037037035, 0.37735849056603776, 0.3333333333333333, 0.425531914893617, 0.4166666666666667, 0.3333333333333333, 0.30303030303030304, 0.4081632653061224, 0.2597402597402597, 0.2898550724637681]
[0.273972602739726, 0.30303030303030304, 0.24390243902439027, 0.29850746268656714, 0.2898550724637681, 0.29411764705882354, 0.29850746268656714, 0.3076923076923077, 0.29850746268656714, 0.26666666666666666, 0.273972602739726, 0.26666666666666666, 0.26666666666666666]


In [ ]:
def calc_home_stake(implied_home_prob, implied_away_prob, implied_draw_prob, total_wager):
    total_implied_prob = implied_home_prob + implied_away_prob + implied_draw_prob
    scaling_factor = total_wager / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    home_stake = round(home_stake, 2)
    
    return home_stake

def calc_away_stake(implied_home_prob, implied_away_prob, implied_draw_prob, total_wager):
    total_implied_prob = implied_home_prob + implied_away_prob + implied_draw_prob
    scaling_factor = total_wager / total_implied_prob
    away_stake = implied_away_prob * scaling_factor
    away_stake = round(away_stake, 2)
    
    return away_stake

def calc_draw_stake(implied_home_prob, implied_away_prob, implied_draw_prob, total_wager):
    total_implied_prob = implied_home_prob + implied_away_prob + implied_draw_prob
    scaling_factor = total_wager / total_implied_prob
    draw_stake = implied_draw_prob * scaling_factor
    draw_stake = round(draw_stake, 2)
    
    return draw_stake


#print(calc_home_stake(0.7462686567164178, 0.28169014084507044, 100))


In [ ]:
#calculating optimal percentage profit (guaranteed):

def calc_opt_perc_profit(implied_home_prob, implied_away_prob, implied_draw_prob):
    total_implied_prob = implied_home_prob + implied_away_prob + implied_draw_prob
    scaling_factor = 1 / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    away_stake = implied_away_prob * scaling_factor
    draw_stake = implied_draw_prob * scaling_factor
    home_payout = home_stake * (1 / implied_home_prob)
    away_payout = away_stake * (1 / implied_away_prob)
    draw_payout = draw_stake * (1 / implied_draw_prob)
    guaranteed_profit = max(home_payout, away_payout, draw_payout) - 1
    percentage_profit = round(guaranteed_profit * 100, 2)
    return f"{percentage_profit}%"

def calc_dollar_profit(implied_home_prob, implied_away_prob, implied_draw_prob):
    total_implied_prob = implied_home_prob + implied_away_prob + implied_draw_prob
    scaling_factor = 1 / total_implied_prob
    home_stake = implied_home_prob * scaling_factor
    away_stake = implied_away_prob * scaling_factor
    draw_stake = implied_draw_prob * scaling_factor
    home_payout = home_stake * (1 / implied_home_prob)
    away_payout = away_stake * (1 / implied_away_prob)
    draw_payout = draw_stake * (1 / implied_draw_prob)
    guaranteed_profit = max(home_payout, away_payout, draw_payout) - 1
    dollar_profit = round(guaranteed_profit * 100, 2)
    return f"${dollar_profit}"

  


In [ ]:
#loop through the home and away OT odds to see if any are arb opps: 
    # append all info to new lists if are 

ot_arb_away_team = []
ot_arb_home_team = []
ot_arb_away_odds = []
ot_arb_home_odds = []
ot_arb_draw_odds = []
ot_arb_away_site = []
ot_arb_home_site = []
ot_arb_draw_site = []
ot_arb_away_wager_amount = []
ot_arb_home_wager_amount = []
ot_arb_draw_wager_amount = []
ot_arb_perc_profit = []
ot_arb_dollar_profit = []

# Ensure both lists have the same length
if len(ot_away_probablity_odds) == len(ot_home_probablity_odds):
    for i in range(len(ot_away_probablity_odds)):
        if ot_away_probablity_odds[i] + ot_home_probablity_odds[i] < 1.1:
            ot_arb_away_team.append(ot_away_teams_list[i])
            ot_arb_home_team.append(ot_home_teams_list[i])
            ot_arb_away_odds.append(ot_best_away_odds_us[i])
            ot_arb_home_odds.append(ot_best_home_odds_us[i])
            ot_arb_draw_odds.append(ot_best_draw_odds_us[i])
            ot_arb_away_site.append(ot_best_away_sites[i])
            ot_arb_home_site.append(ot_best_home_sites[i])
            ot_arb_draw_site.append(ot_best_draw_sites[i])
            ot_arb_away_wager_amount.append(calc_away_stake(ot_home_probablity_odds[i], ot_away_probablity_odds[i], ot_draw_probablility_odds[i], 100))
            ot_arb_home_wager_amount.append(calc_home_stake(ot_home_probablity_odds[i], ot_away_probablity_odds[i], ot_draw_probablility_odds[i], 100))
            ot_arb_draw_wager_amount.append(calc_draw_stake(ot_home_probablity_odds[i], ot_away_probablity_odds[i], ot_draw_probablility_odds[i], 100))
            ot_arb_perc_profit.append(calc_opt_perc_profit(ot_home_probablity_odds[i], ot_away_probablity_odds[i], ot_draw_probablility_odds[i]))
            ot_arb_dollar_profit.append(calc_dollar_profit(ot_home_probablity_odds[i], ot_away_probablity_odds[i], ot_draw_probablility_odds[i]))
        else: 
            print("No Current Arbitrage Opportunities") 


print(ot_best_away_odds_us) 
print(ot_best_home_odds_us) 
print(ot_best_draw_odds_us)


['+140', '+145', '-155', '+140', '+140', '+120', '+175', '+175', '+120', '-105', '+145', '-130', '-115']
['+150', '+165', '+340', '+170', '+165', '+200', '+135', '+140', '+200', '+230', '+145', '+285', '+245']
['+265', '+230', '+310', '+235', '+245', '+240', '+235', '+225', '+235', '+275', '+265', '+275', '+275']


In [ ]:
#making a dict for all the essential arb info 

arb_opps_info = {"Home Team": ot_arb_home_team, "Away Team": ot_arb_away_team, "Home Odds": ot_arb_home_odds, "Away Odds": ot_arb_away_odds, "Draw Odds": ot_arb_draw_odds, "Home Book": ot_arb_home_site, "Away Book": ot_arb_away_site, "Draw Book": ot_arb_draw_site, "Home wager": ot_arb_home_wager_amount, "Away wager": ot_arb_away_wager_amount, "Draw wager": ot_arb_draw_wager_amount, "Profit(%)": ot_arb_perc_profit, "Profit($)": ot_arb_dollar_profit}
print(arb_opps_info)

{'Home Team': ['Los Angeles FC', 'Philadelphia Union', 'Chicago Fire', 'Nashville SC', 'New York City FC', 'Columbus Crew', 'FC Cincinnati', 'Real Salt Lake', 'Seattle Sounders', 'Minnesota United FC', 'Inter Miami CF', 'Houston Dynamo', 'Sporting Kansas City'], 'Away Team': ['Atlanta United FC', 'Charlotte FC', 'DC United', 'CF Montreal', 'New England Revolution', 'Orlando City SC', 'Toronto FC', 'FC Dallas', 'Saint Louis City SC', 'Colorado Rapids', 'Vancouver Whitecaps FC', 'Los Angeles Galaxy', 'Portland Timbers'], 'Home Odds': ['+150', '+165', '+340', '+170', '+165', '+200', '+135', '+140', '+200', '+230', '+145', '+285', '+245'], 'Away Odds': ['+140', '+145', '-155', '+140', '+140', '+120', '+175', '+175', '+120', '-105', '+145', '-130', '-115'], 'Draw Odds': ['+265', '+230', '+310', '+235', '+245', '+240', '+235', '+225', '+235', '+275', '+265', '+275', '+275'], 'Home Book': ['DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'DraftKings', 'Draft

In [ ]:
#making df for arb opps info
df_arb = pd.DataFrame.from_dict(arb_opps_info)
df_arb


,Home Team,Away Team,Home Odds,Away Odds,Draw Odds,Home Book,Away Book,Draw Book,Home wager,Away wager,Draw wager,Profit(%),Profit($)
0,Los Angeles FC,Atlanta United FC,+150,+140,+265,DraftKings,DraftKings,DraftKings,36.68,38.20,25.12,-8.31%,$-8.31
1,Philadelphia Union,Charlotte FC,+165,+145,+230,DraftKings,DraftKings,DraftKings,34.67,37.50,27.84,-8.13%,$-8.13
2,Chicago Fire,DC United,+340,-155,+310,DraftKings,DraftKings,DraftKings,21.10,56.26,22.64,-7.17%,$-7.17
3,Nashville SC,CF Montreal,+170,+140,+235,DraftKings,DraftKings,DraftKings,34.12,38.38,27.50,-7.88%,$-7.88
4,New York City FC,New England Revolution,+165,+140,+245,DraftKings,DraftKings,DraftKings,34.82,38.44,26.74,-7.74%,$-7.74
5,Columbus Crew,Orlando City SC,+200,+120,+240,DraftKings,DraftKings,DraftKings,30.81,42.01,27.18,-7.58%,$-7.58
6,FC Cincinnati,Toronto FC,+135,+175,+235,DraftKings,DraftKings,DraftKings,39.12,33.43,27.44,-8.06%,$-8.06
7,Real Salt Lake,FC Dallas,+140,+175,+225,DraftKings,DraftKings,DraftKings,38.30,33.42,28.28,-8.09%,$-8.09
8,Seattle Sounders,Saint Louis City SC,+200,+120,+235,DraftKings,DraftKings,DraftKings,30.68,41.84,27.48,-7.95%,$-7.95
9,Minnesota United FC,Colorado Rapids,+230,-105,+275,DraftKings,DraftKings,DraftKings,27.99,47.37,24.63,-7.62%,$-7.62


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


def send_email(x):
    MY_ADDRESS = "surebetsignals@outlook.com"
    MY_PASSWORD = "Maddie2013$"
    RECIPIENT_ADDRESS = "dillonkercher@icloud.com"

    HOST_ADDRESS = "smtp-mail.outlook.com"
    HOST_PORT = 587

    # Set up the SMTP server
    server = smtplib.SMTP(host=HOST_ADDRESS, port=HOST_PORT)
    server.starttls()
    server.login(MY_ADDRESS, MY_PASSWORD)

    # Create the email
    message = MIMEMultipart()
    message["From"] = MY_ADDRESS
    message["To"] = RECIPIENT_ADDRESS
    message["Subject"] = "Arbitrage Opportunity"

# Convert DataFrame to HTML table
    html_table = x.to_html(index=False)

    # Attach the HTML table to the email body
    htmlPart = MIMEText(html_table, 'html')
    message.attach(htmlPart)

    # Send the email
    server.send_message(message)
    server.quit()

    print("email sent successfully")

# Send email if condition is met
def check_condition():
    if ot_away_probablity_odds[i] + ot_home_probablity_odds[i] < 1.1:
        send_email(df_arb)
    else: print("No current abritrage opportunities")

if __name__ == "__main__":
    check_condition()

    


email sent successfully
